In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedShuffleSplit
from IPython.display import clear_output

import tensorflow as tf

In [2]:
# Constants
start = 0
end = 10_000_000
log_frequency = 10
batch_size = 1024
epochs = 100
width = 15

In [ ]:
def int_to_digit_array(arr, width=15):
    powers_of_ten = 10 ** np.arange(width)[::-1]
    digit_array = (arr[:, np.newaxis] // powers_of_ten) % 10
    return digit_array

def data_generator(start, end, batch_size, width):
    while start < end:
        batch_indices = np.arange(start, min(start + batch_size, end))
        X_batch = int_to_digit_array(batch_indices, width)
        y_batch = np.column_stack((batch_indices % 2, (batch_indices + 1) % 2))
        yield X_batch, y_batch
        start += batch_size

train_dataset = tf.data.Dataset.from_generator(
    lambda: data_generator(start, end, batch_size, width),
    output_signature=(
        tf.TensorSpec(shape=(None, width), dtype=tf.int32),
        tf.TensorSpec(shape=(None, 2), dtype=tf.int32)
    )
).shuffle(buffer_size=10000).prefetch(tf.data.AUTOTUNE)


split_train = StratifiedShuffleSplit(n_splits=1, test_size=0.4, train_size=0.6)
all_indices = np.arange(start, end)
y_full = np.column_stack((all_indices % 2, (all_indices + 1) % 2))

train_index, test_index = next(split_train.split(all_indices, y_full))

val_dataset = tf.data.Dataset.from_generator(
    lambda: data_generator(all_indices[test_index][0], all_indices[test_index][-1] + 1, batch_size, width),
    output_signature=(
        tf.TensorSpec(shape=(None, width), dtype=tf.int32),
        tf.TensorSpec(shape=(None, 2), dtype=tf.int32)
    )
).prefetch(tf.data.AUTOTUNE)

In [5]:
class CustomLoggingCallback(tf.keras.callbacks.Callback):
    def __init__(self, log_frequency):
        super(CustomLoggingCallback, self).__init__()
        self.log_frequency = log_frequency

    def on_epoch_end(self, epoch, logs=None):
        if (epoch + 1) % self.log_frequency == 0:
            print(f'Epoch {epoch + 1}: {logs}')

In [6]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(50, activation='relu'),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(70, activation='tanh'),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(50, activation='relu'),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(50, activation='sigmoid'),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(50, activation='relu'),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(2, activation='softmax')
])

In [7]:
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=['accuracy'])

In [8]:
#model.evaluate(X_train, y_train)

In [9]:
#model.evaluate(X_val, y_val)

In [10]:
custom_callback = CustomLoggingCallback(log_frequency)

model.fit(train_dataset, epochs=epochs, validation_data=val_dataset, callbacks=[custom_callback] ,verbose=1)

Epoch 1/100
5860/5860 ━━━━━━━━━━━━━━━━━━━━ 70s 10ms/step - accuracy: 0.7756 - loss: 0.3616 - val_accuracy: 1.0000 - val_loss: 2.5878e-04
Epoch 2/100
5860/5860 ━━━━━━━━━━━━━━━━━━━━ 41s 6ms/step - accuracy: 0.9967 - loss: 0.0125 - val_accuracy: 1.0000 - val_loss: 9.5738e-05
Epoch 3/100
5860/5860 ━━━━━━━━━━━━━━━━━━━━ 55s 9ms/step - accuracy: 0.9980 - loss: 0.0082 - val_accuracy: 1.0000 - val_loss: 6.2090e-05
Epoch 4/100
1276/5860 ━━━━━━━━━━━━━━━━━━━━ 23s 5ms/step - accuracy: 0.9984 - loss: 0.0066

KeyboardInterrupt: 

In [ ]:
plt.style.use('dark_background')

plt.plot(model.history.history['accuracy'])
plt.plot(model.history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

plt.plot(model.history.history['loss'])
plt.plot(model.history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
print(f"maximum accurary of '{max(model.history.history['val_accuracy'])}' at epoch number {model.history.history['val_accuracy'].index(max(model.history.history['val_accuracy']))}")
print(f"mimimum loss of '{min(model.history.history['val_loss'])}' at epoch number {model.history.history['val_loss'].index(min(model.history.history['val_loss']))}")

In [13]:
model.save('model.keras')

In [14]:
def num_to_nparray(num):
    return np.array([list(map(int, str(num).zfill(15)))])

In [15]:
def predict_odd(num):
    prediction = model.predict(num_to_nparray(num))
    return prediction[0][0] > prediction[0][1]

In [16]:
#model.predict(num_to_nparray(32))

In [17]:
#predict_odd(33)

In [18]:
#import_model = tf.keras.models.load_model('model.keras')

In [19]:
#import_model.predict(num_to_nparray(100006))

### Testing


In [21]:
model = tf.keras.models.load_model('model.keras')
def num_to_nparray(nums):
    return np.array([list(map(int, str(num).zfill(15))) for num in nums])

def predict_odd_batch(nums):
    predictions = model.predict(num_to_nparray(nums), verbose=0)
    return predictions[:, 0] > predictions[:, 1]

batch_size = 1000
predictions = []
for i in range(1_000_000_000_000_000):
    predictions.append(i)

    if len(predictions) == batch_size:
        predict_results = predict_odd_batch(predictions)
        for j, num in enumerate(predictions):
            correct = bool(num % 2)
            if predict_results[j] != correct:
                print(f"Error at number {num}, should be {correct}, predicted {predict_results[j]}")
                break
        predictions.clear()

    if i % 10_000_000 == 0:
        print(f"At number {i}")
    if i % 50_000_000 == 0:
        clear_output(wait=True)
        print(f"At number {i}")

if predictions:
    predict_results = predict_odd_batch(predictions)
    for j, num in enumerate(predictions):
        correct = bool(num % 2)
        if predict_results[j] != correct:
            print(f"Error at number {num}, should be {correct}, predicted {predict_results[j]}")

At number 0
Error at number 100002, should be False, predicted True
Error at number 101002, should be False, predicted True
Error at number 102002, should be False, predicted True
Error at number 103002, should be False, predicted True
Error at number 104002, should be False, predicted True
Error at number 105002, should be False, predicted True
Error at number 106002, should be False, predicted True
Error at number 107002, should be False, predicted True
Error at number 108002, should be False, predicted True
Error at number 109002, should be False, predicted True
Error at number 110002, should be False, predicted True
Error at number 111002, should be False, predicted True
Error at number 112002, should be False, predicted True
Error at number 113002, should be False, predicted True
Error at number 114002, should be False, predicted True
Error at number 115002, should be False, predicted True
Error at number 116002, should be False, predicted True
Error at number 117002, should be Fa

KeyboardInterrupt: 